In [70]:
from transformers import BertTokenizer,AutoTokenizer
from konlpy.tag import Mecab
from datasets import load_from_disk
from IPython.core.display import HTML
import re

In [71]:
#load data
dataset = load_from_disk("../../data/train_dataset")
trainset = dataset['train']
validset = dataset['validation']

In [72]:
# trainset['answers'][0]['text'][0]
trainset['question'][0]

'대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'

In [73]:
#load tokenizer
checkpoint = "klue/roberta-large"
tokenizer = BertTokenizer.from_pretrained(checkpoint, use_fast=True)

#AutoTokenizer로 하면 Vocab 수정이 불가함
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)


#load mecab
m = Mecab()


In [74]:
def preprocesing_context(data):
    return re.sub("(\\n)+|(\\\\n)+|(\xa0)|(\u3000)","",data)
    # context = re.sub(r"\\n"," ",context) #모든 data
    # context = re.sub(r"\'","",context) #22 특정 문자열 강조를 위해 '문자열'가 사용될 때 \'문자열\' 형태로 입력이 되어 있어서 \를 제거
    # context = re.sub("[a-zA-Z가-힣 ]+(\|)[a-zA-Z가-힣]+(\|)[a-zA-Z가-힣 ]+(.)"," ",context) #27 섬네일|left|페데리코 다 몬테펠트로의 구비오 스투디올로의 모습. 제거
    ######################### context = re.sub("(\|)[a-zA-Z가-힣]+(\|)"," ",context) #33 |비트겐슈타인| 과 같은 문자열 제거
    #필수
    # context = re.sub("p=(\d)+\-(\d)+|p=(\d)+","",context) #37 page 제거
    #필수
    # context = re.sub("(된다)(.){0}","된다.",context) #43 된다 => 된다.
    ######################### context = re.sub("(\*)*","",context) #43,50 "*" => ""
    # context = re.sub("( )+", " ", context)

    # 애매함
    """
        유지 => 51 : 1942 ~ 43 == 메이저 리그로 ㅣ입단 ==
        테스트 => 51 : .414
        유지 => 52 : 교리·이조좌랑·사간·부응교·종부시정·이조참의·동부승지·좌부승지·우승지·남양부사·부제학·대사간·좌승지·수찬관·대사성·병조참의·도승지·강원도관찰사·한성부우윤·대사헌·예문관제학·대제학·이조판서·우빈객·좌빈객

    """
    return context
#53번부터
i=1466
print(preprocesing_context(trainset[i]['context']),"\n")

나리타공항 건설에서 화근이 된 일본 정부의 정책과 그로 인해 발생한 산리즈카 투쟁은 양측에 모두 비참하기만 한 결과를 가져왔다. 이후 공공사업을 둘러싸고 분쟁이 일어나는 현장에서는 “합의형성의 노력을 하지 않은 채 힘에 의지해 사업을 진행하면 힘에 의한 저항을 낳는다”는 자숙을담아 “나리타처럼 되지 말자(成田のようにならないようにしよう )”가 표어가 되었다. 내륙에 공항을 건설하면 토지취득 및 소음 문제가 현저하게 발생할 수밖에 없는데, 나리타에서는 산리즈카 투쟁까지 더해져 막대한 손실을 초래했다. 이 교훈으로 이후 일본은 해상 및 원격지에 공항을 건설하는 경향이 커졌고, 이는 일본 공항의 편리성 저하와 비용 증가로 이어졌다.group=주해|그러나 해상에 건설된 간사이 국제공항에도 중핵파가 공항건설공사의 견학선에 방화하는 테러사건(요코하마요트 소형여객선 폭파사건)을 일으킨 바 있다. group=주해|간사이공항 인공섬 건설 총 공사비가 1조 엔임을 처음 들은 세계 공항관계자는 이 막대한 건설비에 통역의 오역이 아닌지 반문하기도 했다. 또한 토지수용과 행정대집행도 신중하게 실시되도록 되었으니, 산리즈카 투쟁은 일본 공공사업 전반에 엄청난 영향을 미친 것이다 나리타 하늘과 대지의 역사관은 국가공무원 종합직 초임자와 나리타공항회사 신입사원 연수에 포함되어 있다. 제2의 나리타를 만들지 않도록 산리즈카 투쟁의 교훈을 살리기 위한 노력이라 한다 한편 개발에 반대하는 주민 측에서도 실력투쟁에서 주민투표 등으로 투쟁방식을 바꾸는 전환점이 되었다 특히 장기간에 걸친 투쟁을 거쳐 이루어진 나리타 공항 문제 심포지엄・나리타 공항 문제 원탁회의는 이후 공공사업 실시 시의 모델 케이스가 되어, 나가라강 하구둑 문제도 원탁회의 방식이 채택되었고, 얀바댐 반대운동을 하던 주민들이 공부하러 나리타 심포지엄을 방문하는 등 일본 전국 각지에서 행해지던 주민운동의 참고사례가 되었다. 관제탑 점거사건 등은 일본 국외에도 크게 보도되어 산리즈카 투쟁의 모습은 영화나 음악 등 예술작품의 테마로 사용되

In [75]:
def word_highligt_html(word, color='black', highlight='lightblue'):
    color_word = f'<span style="color: {color}; background-color:{highlight}"><b>{word}</b></span>'
    return color_word

In [76]:
def restore_word_by_tokens(tokenized_tokens):
    word = ""
    for token in tokenized_tokens:
        if "##" in token: # ##이 붙은 경우 ##을 떼고 입력
            word +=re.sub("##","",token) 
        elif "[UNK]"==token or re.sub("[a-zA-Z0-9가-힣]","",token)!="": #특수문자나 다른 나라 언어인 경우 그대로 입력
            word += token 
        else: #나머지 경우는 앞의 한 칸을 띄고 입력
            word +=" " + token
    return word.strip() #양쪽 빈칸 제거


In [77]:
chn_char_stt = int("4E00",16) #한자 유니코드 
chn_char_end = int("9FFF",16) #한자 유니코드
added_token_set = set()
unk_sent = []
for i, answer in enumerate(trainset['answers']):
    answer_text = answer['text'][0]
    
    # mecab 형태소 분류기로 answer 분리
    tokens_mecab = m.morphs(answer_text)
    tokens_bert = []

    # 분리된 answer를 Berttokenizer로 subword 단위로 분리
    for token_mecab in tokens_mecab:
        tokens_bert.extend(tokenizer.tokenize(token_mecab))
    
    # # UNK가 있는 sequence가 있는 경우에만 진행
    if "[UNK]" in tokens_bert:
        unk_sent.append(answer_text)
        token_flag = False #Flag=False면 그대로, True 이후 음절은 ## 추가
        resotred_answer = restore_word_by_tokens(tokens_bert) #Mecab-BertTokenzer를 통과한 subword를 문장으로 복원, 단 UNK 존재함
        for s in restore_word_by_tokens(tokens_mecab) : #Mecab으로 분류된 형태소 토큰을 문장으로 복원
            if s in resotred_answer: #음절 단위로 비교, 문장 내에 존재하면 pass
                token_flag = False
                continue
            
            # try: #한자 인 경우
            if chn_char_stt <= ord(s) <= chn_char_end: # UNK인 subword가 한자인 경우 추가
                added_token_set.add(s)
            # except TypeError: #한자가 아닌 경우
            elif token_flag == False:
                added_token_set.add(s)
                token_flag = True
            else:
                added_token_set.add("##"+s)

In [87]:
for i,sent in enumerate(trainset['question']):
    if "[UNK]" in tokenizer.tokenize(sent):
        break


In [ ]:
m.morphs(answer_text)

In [91]:
print(trainset['question'][i])
print(tokenizer.tokenize(trainset['question'][i]))

홀뮴이 평소와 다르게 강자성을 가지게 되는 온도는?
['[UNK]', '평소', '##와', '다르', '##게', '강자', '##성', '##을', '가지', '##게', '되', '##는', '온도', '##는', '?']


['크리스', '토포', '알', '하우스']

In [78]:
from collections import deque
unused_vocab_list = deque()
for v,i in tokenizer.vocab.items():
    if "unused" in v:
        unused_vocab_list.append((v,i))

In [79]:
for added_token in added_token_set:
    if added_token not in tokenizer.vocab.keys():
        del_key, num = unused_vocab_list.popleft()
        del tokenizer.vocab[del_key]
        tokenizer.vocab[added_token] = num
print("Done")

Done


In [80]:
print({v:i for v,i in sorted(tokenizer.vocab.items(),key=lambda x : x[1]) if 31500<=i<31600})

{'[unused0]': 31500, '[unused1]': 31501, '[unused2]': 31502, '[unused3]': 31503, '[unused4]': 31504, '[unused5]': 31505, '[unused6]': 31506, '[unused7]': 31507, '[unused8]': 31508, '[unused9]': 31509, '[unused10]': 31510, '[unused11]': 31511, '[unused12]': 31512, '[unused13]': 31513, '[unused14]': 31514, '[unused15]': 31515, '[unused16]': 31516, '[unused17]': 31517, '[unused18]': 31518, '[unused19]': 31519, '[unused20]': 31520, '[unused21]': 31521, '[unused22]': 31522, '[unused23]': 31523, '[unused24]': 31524, '[unused25]': 31525, '[unused26]': 31526, '[unused27]': 31527, '[unused28]': 31528, '[unused29]': 31529, '[unused30]': 31530, '[unused31]': 31531, '[unused32]': 31532, '[unused33]': 31533, '[unused34]': 31534, '[unused35]': 31535, '[unused36]': 31536, '[unused37]': 31537, '[unused38]': 31538, '[unused39]': 31539, '[unused40]': 31540, '[unused41]': 31541, '[unused42]': 31542, '[unused43]': 31543, '[unused44]': 31544, '[unused45]': 31545, '[unused46]': 31546, '[unused47]': 31547, '

In [11]:
tokenizer.save_pretrained("../robertatokenizer_customized")

('../robertatokenizer_customized/tokenizer_config.json',
 '../robertatokenizer_customized/special_tokens_map.json',
 '../robertatokenizer_customized/vocab.txt',
 '../robertatokenizer_customized/added_tokens.json')

In [12]:
tokenizer = BertTokenizer.from_pretrained("../robertatokenizer_customized")

In [13]:
chn_char_stt = int("4E00",16) #한자 유니코드 
chn_char_end = int("9FFF",16) #한자 유니코드
added_token_set = set()
unk_sent = []
for i, answer in enumerate(trainset['answers']):
    answer_text = answer['text'][0]
    
    # mecab 형태소 분류기로 answer 분리
    tokens_mecab = m.morphs(answer_text)
    tokens_bert = []

    # 분리된 answer를 Berttokenizer로 subword 단위로 분리
    for token_mecab in tokens_mecab:
        tokens_bert.extend(tokenizer.tokenize(token_mecab))
    
    # # UNK가 있는 sequence가 있는 경우에만 진행
    if "[UNK]" in tokens_bert:
        unk_sent.append(answer_text)
        token_flag = False #Flag=False면 그대로, True 이후 음절은 ## 추가
        resotred_answer = restore_word_by_tokens(tokens_bert) #Mecab-BertTokenzer를 통과한 subword를 문장으로 복원, 단 UNK 존재함
        for s in restore_word_by_tokens(tokens_mecab) : #Mecab으로 분류된 형태소 토큰을 문장으로 복원
            if s in resotred_answer: #음절 단위로 비교, 문장 내에 존재하면 pass
                token_flag = False
                continue
            
            # try: #한자 인 경우
            if chn_char_stt <= ord(s) <= chn_char_end: # UNK인 subword가 한자인 경우 추가
                added_token_set.add(s)
            # except TypeError: #한자가 아닌 경우
            elif token_flag == False:
                added_token_set.add(s)
                token_flag = True
            else:
                added_token_set.add("##"+s)
print(len(added_token_set))

1
